# rf201_composite
Addition and convolution: composite pdf with signal and background component

```
 pdf = f_bkg * bkg(x,a0,a1) + (1-fbkg) * (f_sig1 * sig1(x,m,s1 + (1-f_sig1) * sig2(x,m,s2)))
```




**Author:** Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Sunday, December 01, 2024 at 10:32 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooChebychev.h"
#include "RooAddPdf.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"
using namespace RooFit;

Setup component pdfs
---------------------------------------

Declare observable x

In [2]:
RooRealVar x("x", "x", 0, 10);

Create two Gaussian PDFs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and their parameters

In [3]:
RooRealVar mean("mean", "mean of gaussians", 5);
RooRealVar sigma1("sigma1", "width of gaussians", 0.5);
RooRealVar sigma2("sigma2", "width of gaussians", 1);

RooGaussian sig1("sig1", "Signal component 1", x, mean, sigma1);
RooGaussian sig2("sig2", "Signal component 2", x, mean, sigma2);

[#0] WARNING:InputArguments -- The parameter 'sigma1' with range [-inf, inf] of the RooGaussian 'sig1' exceeds the safe range of (0, inf). Advise to limit its range.
[#0] WARNING:InputArguments -- The parameter 'sigma2' with range [-inf, inf] of the RooGaussian 'sig2' exceeds the safe range of (0, inf). Advise to limit its range.


Build Chebychev polynomial pdf

In [4]:
RooRealVar a0("a0", "a0", 0.5, 0., 1.);
RooRealVar a1("a1", "a1", 0.2, 0., 1.);
RooChebychev bkg("bkg", "Background", x, RooArgSet(a0, a1));

---------------------------------------------
METHOD 1-Two RooAddPdfs
=============================================

Add signal components
------------------------------------------

Sum the signal components into a composite signal pdf

In [5]:
RooRealVar sig1frac("sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.);
RooAddPdf sig("sig", "Signal", RooArgList(sig1, sig2), sig1frac);

Add signal and background
------------------------------------------------

Sum the composite signal and background

In [6]:
RooRealVar bkgfrac("bkgfrac", "fraction of background", 0.5, 0., 1.);
RooAddPdf model("model", "g1+g2+a", RooArgList(bkg, sig), bkgfrac);

Sample, fit and plot model
---------------------------------------------------

Generate a data sample of 1000 events in x from model

In [7]:
std::unique_ptr<RooDataSet> data{model.generate(x, 1000)};

input_line_55:2:2: warning: 'data' shadows a declaration with the same name in the 'std' namespace; use '::data' to reference this declaration
 std::unique_ptr<RooDataSet> data{model.generate(x, 1000)};
 ^


Fit model to data

In [8]:
model.fitTo(*data, PrintLevel(-1));

input_line_56:2:15: error: reference to 'data' is ambiguous
 model.fitTo(*data, PrintLevel(-1));
              ^
input_line_55:2:30: note: candidate found by name lookup is 'data'
 std::unique_ptr<RooDataSet> data{model.generate(x, 1000)};
                             ^
/usr/include/c++/9/bits/range_access.h:318:5: note: candidate found by name lookup is 'std::data'
    data(initializer_list<_Tp> __il) noexcept
    ^
/usr/include/c++/9/bits/range_access.h:289:5: note: candidate found by name lookup is 'std::data'
    data(_Container& __cont) noexcept(noexcept(__cont.data()))
    ^
/usr/include/c++/9/bits/range_access.h:299:5: note: candidate found by name lookup is 'std::data'
    data(const _Container& __cont) noexcept(noexcept(__cont.data()))
    ^
/usr/include/c++/9/bits/range_access.h:309:5: note: candidate found by name lookup is 'std::data'
    data(_Tp (&__array)[_Nm]) noexcept
    ^


Plot data and PDF overlaid

In [9]:
RooPlot *xframe = x.frame(Title("Example of composite pdf=(sig1+sig2)+bkg"));
data->plotOn(xframe);
model.plotOn(xframe);

input_line_57:3:1: error: reference to 'data' is ambiguous
data->plotOn(xframe);
^
input_line_55:2:30: note: candidate found by name lookup is 'data'
 std::unique_ptr<RooDataSet> data{model.generate(x, 1000)};
                             ^
/usr/include/c++/9/bits/range_access.h:318:5: note: candidate found by name lookup is 'std::data'
    data(initializer_list<_Tp> __il) noexcept
    ^
/usr/include/c++/9/bits/range_access.h:289:5: note: candidate found by name lookup is 'std::data'
    data(_Container& __cont) noexcept(noexcept(__cont.data()))
    ^
/usr/include/c++/9/bits/range_access.h:299:5: note: candidate found by name lookup is 'std::data'
    data(const _Container& __cont) noexcept(noexcept(__cont.data()))
    ^
/usr/include/c++/9/bits/range_access.h:309:5: note: candidate found by name lookup is 'std::data'
    data(_Tp (&__array)[_Nm]) noexcept
    ^


Overlay the background component of model with a dashed line

In [10]:
model.plotOn(xframe, Components(bkg), LineStyle(kDashed));

input_line_58:2:50: error: cannot take the address of an rvalue of type 'ELineStyle'
 model.plotOn(xframe, Components(bkg), LineStyle(kDashed));
                                                 ^~~~~~~
Error while creating dynamic expression for:
  model.plotOn(xframe, Components(bkg), LineStyle(kDashed))


Overlay the background+sig2 components of model with a dotted line

In [11]:
model.plotOn(xframe, Components(RooArgSet(bkg, sig2)), LineStyle(kDotted));

input_line_59:2:67: error: cannot take the address of an rvalue of type 'ELineStyle'
 model.plotOn(xframe, Components(RooArgSet(bkg, sig2)), LineStyle(kDotted));
                                                                  ^~~~~~~
Error while creating dynamic expression for:
  model.plotOn(xframe, Components(RooArgSet(bkg, sig2)), LineStyle(kDotted))


Print structure of composite pdf

In [12]:
model.Print("t");

0x7fd4f12293e8 RooAddPdf::model = 0.9/1 [Auto,Clean] 
  0x7fd4f80037d0/V- RooChebychev::bkg = 0.8 [Auto,Dirty] 
    0x7fd4f801c000/V- RooRealVar::x = 5
    0x7fd4f8003000/V- RooRealVar::a0 = 0.5
    0x7fd4f80033e8/V- RooRealVar::a1 = 0.2
  0x7fd4f1229000/V- RooRealVar::bkgfrac = 0.5
  0x7fd4f122c3e8/V- RooAddPdf::sig = 1/1 [Auto,Clean] 
    0x7fd4f8006bb8/V- RooGaussian::sig1 = 1 [Auto,Dirty] 
      0x7fd4f801c000/V- RooRealVar::x = 5
      0x7fd4f8006000/V- RooRealVar::mean = 5
      0x7fd4f80063e8/V- RooRealVar::sigma1 = 0.5
    0x7fd4f122c000/V- RooRealVar::sig1frac = 0.8
    0x7fd4f8007110/V- RooGaussian::sig2 = 1 [Auto,Dirty] 
      0x7fd4f801c000/V- RooRealVar::x = 5
      0x7fd4f8006000/V- RooRealVar::mean = 5
      0x7fd4f80067d0/V- RooRealVar::sigma2 = 1


---------------------------------------------------------------------------------------------
METHOD 2-One RooAddPdf with recursive fractions
=============================================================================================

Construct sum of models on one go using recursive fraction interpretations

model2 = bkg + (sig1 + sig2)

In [13]:
RooAddPdf model2("model", "g1+g2+a", RooArgList(bkg, sig1, sig2), RooArgList(bkgfrac, sig1frac), true);

NB: Each coefficient is interpreted as the fraction of the
left-hand component of the i-th recursive sum, i.e.

sum4 = A + ( B + ( C + D)  with fraction fA, fB and fC expands to

sum4 = fA*A + (1-fA)*(fB*B + (1-fB)*(fC*C + (1-fC)*D))

Plot recursive addition model
---------------------------------------------------------

In [14]:
model2.plotOn(xframe, LineColor(kRed), LineStyle(kDashed));
model2.plotOn(xframe, Components(RooArgSet(bkg, sig2)), LineColor(kRed), LineStyle(kDashed));
model2.Print("t");

input_line_62:2:34: error: cannot take the address of an rvalue of type 'EColor'
 model2.plotOn(xframe, LineColor(kRed), LineStyle(kDashed));
                                 ^~~~
Error while creating dynamic expression for:
  model2.plotOn(xframe, LineColor(kRed), LineStyle(kDashed))
input_line_62:3:67: error: cannot take the address of an rvalue of type 'EColor'
model2.plotOn(xframe, Components(RooArgSet(bkg, sig2)), LineColor(kRed), LineStyle(kDashed));
                                                                  ^~~~
Error while creating dynamic expression for:
  model2.plotOn(xframe, Components(RooArgSet(bkg, sig2)), LineColor(kRed), LineStyle(kDashed))


Draw the frame on the canvas

In [15]:
new TCanvas("rf201_composite", "rf201_composite", 600, 600);
gPad->SetLeftMargin(0.15);
xframe->GetYaxis()->SetTitleOffset(1.4);
xframe->Draw();

IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime8internal9EvaluateTIvEET_PNS1_15DynamicExprInfoEPN5clang11DeclContextE' unresolved while linking [cling interface function]!
You are probably missing the definition of void cling::runtime::internal::EvaluateT<void>(cling::runtime::internal::DynamicExprInfo*, clang::DeclContext*)
Maybe you need to load the corresponding shared library?


Draw all canvases 

In [16]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()